In [3]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.3'
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [75.2 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,604 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,696 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis

In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('HomeSales')

In [7]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
  SELECT
    YEAR(date) AS year,
    ROUND(AVG(price), 2) as avg_price
  FROM HomeSales
  WHERE bedrooms = 4
  GROUP BY year
  ORDER BY year""").show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [8]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
  SELECT
    ROUND(AVG(price), 2) as avg_price,
    date_built
  FROM HomeSales
  WHERE bedrooms = 4 AND bathrooms = 3
  GROUP BY
    date_built
  ORDER BY
    date_built
""").show()


+---------+----------+
|avg_price|date_built|
+---------+----------+
|287529.97|      2010|
|294656.14|      2011|
|289533.13|      2012|
|291870.89|      2013|
|289935.92|      2014|
| 299517.5|      2015|
|289090.11|      2016|
|289100.17|      2017|
+---------+----------+



In [9]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
  SELECT
    ROUND(AVG(price), 2) as avg_price,
    date_built
  FROM HomeSales
  WHERE bedrooms = 4 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
  GROUP BY
    date_built
  ORDER BY
    date_built
""").show()


+---------+----------+
|avg_price|date_built|
+---------+----------+
|290237.14|      2010|
|293021.78|      2011|
|290024.97|      2012|
|293916.19|      2013|
|306202.48|      2014|
|305769.02|      2015|
|298295.66|      2016|
|289215.12|      2017|
+---------+----------+



In [14]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
spark.sql("""
  SELECT
    ROUND(AVG(price), 2) as avg_price,
    view
  FROM HomeSales
  GROUP BY
    view
  HAVING avg_price >= 350000
  ORDER BY
    view DESC
""").show()

start_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+
| avg_price|view|
+----------+----+
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
| 991767.38|  80|
+----------+----+
only showing top 20 rows

--- 9.655952453613281e-05 seconds ---


In [24]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table HomeSales")


DataFrame[]

In [25]:
# 8. Check if the table is cached.
spark.catalog.isCached('HomeSales')

True

In [26]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
spark.sql("""
  SELECT
    ROUND(AVG(price), 2) as avg_price,
    view
  FROM HomeSales
  GROUP BY
    view
  HAVING avg_price >= 350000
  ORDER BY
    view DESC
""")

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 0.00013065338134765625 seconds ---


Cached time is slightly slower than the temporary table. Really the difference is very low but if the data set were larger I am sure that it would make a big difference in time.

In [27]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

df.write\
  .mode("overwrite")\
  .format("parquet")\
  .partitionBy("date_built")\
  .save("path/to/formatted_parquet_home_sales")

In [28]:
# 11. Read the parquet formatted data.
p_df= spark.read.parquet('path/to/formatted_parquet_home_sales')

In [29]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceGlobalTempView('p_home_sales')

In [32]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.
spark.sql("""
  SELECT
    ROUND(AVG(price), 2) as avg_price,
    view
  FROM global_temp.p_home_sales
  GROUP BY
    view
  HAVING AVG(price) >= 350000
  ORDER BY
    view DESC
""").show()
start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+
| avg_price|view|
+----------+----+
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
| 991767.38|  80|
+----------+----+
only showing top 20 rows

--- 9.799003601074219e-05 seconds ---


In [33]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table HomeSales")

DataFrame[]

In [34]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('HomeSales')

False